In [ ]:
import panel as pn
import holoviews as hv
import panel.widgets as pnw
from pnbkext.lumino.pane.lumino import LuminoDataGrid, LuminoDock
# must be import before calling pn.extension else model will not be register
from bokeh.sampledata.autompg import autompg

if pn.state.curdoc is not None:
    # lumino css is already imported in juyter lab but not if deployed on a bokeh server
    # importing a second time the css make jupyter lab not usable
    import os
    import pnbkext
    with open(os.path.join(pnbkext.__path__[0], 'lumino', 'styles', 'style.css'), 'r') as f:
        lumino_css = f.read()
    pn.extension(raw_css=[lumino_css])
else:
    pn.extension()
hv.extension('bokeh')

In [ ]:
df = autompg.copy()

ORIGINS = ['North America', 'Europe', 'Asia']

# data cleanup
df.origin = [ORIGINS[x-1] for x in df.origin]

df['mfr'] = [x.split()[0] for x in df.name]
df.loc[df.mfr=='chevy', 'mfr'] = 'chevrolet'
df.loc[df.mfr=='chevroelt', 'mfr'] = 'chevrolet'
df.loc[df.mfr=='maxda', 'mfr'] = 'mazda'
df.loc[df.mfr=='mercedes-benz', 'mfr'] = 'mercedes'
df.loc[df.mfr=='toyouta', 'mfr'] = 'toyota'
df.loc[df.mfr=='vokswagen', 'mfr'] = 'volkswagen'
df.loc[df.mfr=='vw', 'mfr'] = 'volkswagen'
del df['name']

columns = sorted(df.columns)
discrete = [x for x in columns if df[x].dtype == object]
continuous = [x for x in columns if x not in discrete]
quantileable = [x for x in continuous if len(df[x].unique()) > 20]

x = pnw.Select(name='X-Axis', value='mpg', options=quantileable)
y = pnw.Select(name='Y-Axis', value='hp', options=quantileable)
size = pnw.Select(name='Size', value='None', options=['None'] + quantileable)
color = pnw.Select(name='Color', value='None', options=['None'] + quantileable)

@pn.depends(x.param.value, y.param.value, color.param.value, size.param.value) 
def create_figure(x, y, color, size):
    opts = dict(cmap='rainbow', line_color='black', responsive=True)
    if color != 'None':
        opts['color'] = color 
    if size != 'None':
        opts['size'] = hv.dim(size).norm()*20
    return hv.Points(df, [x, y], label="%s vs %s" % (x.title(), y.title())).opts(**opts)

widgets = pn.WidgetBox(x, y, color, size, width=200)

In [ ]:
dg = LuminoDataGrid(autompg, sizing_mode='stretch_both')
@pn.depends(dg.param.selections)
def print_selections(selections):
    return pn.pane.JSON(selections)

parameters = pn.Param(dg, parameters=['selection_mode', 'gridstyle', 'row_height', 'column_width'])
parameters_reordered = pn.Column(parameters[0], parameters[1], parameters[4], parameters[2], parameters[3],sizing_mode="stretch_width")

pn.Row(
    LuminoDock(
        ('DataGrid Params', pn.Row(parameters_reordered, print_selections, sizing_mode='stretch_width')),
        ('Plot Params', widgets),
        width=300,
    ),
    LuminoDock(
        ('DataGrid', dg),
        ('Plot', pn.panel(create_figure,  sizing_mode = "stretch_both")),
        sizing_mode="stretch_width",
    ), sizing_mode="stretch_both"
)